In [1]:
import csv
import io
import os
import pickle
import uuid

In [2]:
TRAFFIC_INFO_DIR = "traffic_info_here"
DT_FMT = "%Y-%m-%d_%H:%M"
LIMIT = 100

fmap = {}
for i, entry in enumerate(os.scandir(TRAFFIC_INFO_DIR)):
    dt, ext = os.path.splitext(entry.name)
    if ext != '.pickle':
        continue
    fmap[dt] = entry.path

traffic_info_dict = {}
dt_list = sorted(fmap)[:LIMIT]
for dt in dt_list:
    with io.open(fmap[dt], "rb") as f:
        traffic_info_dict[dt] = pickle.load(f)

# for each timestamp there's a dict that maps node position codes to data records
nodes = set(traffic_info_dict[dt_list[0]])
assert all(set(_) == nodes for _ in traffic_info_dict.values())

In [3]:
date_time = dt_list[0]
print(f"timestamp: {date_time}")
d = traffic_info_dict[dt_list[0]]
{k: v for k, v in sorted(d.items())[:2]}

timestamp: 2019-02-21_12:33


{7604: {'flux': 40.6,
  'length': 0.05632,
  'conf': 0.85,
  'jam': 1.3075,
  'speed': (35.96, 35.96),
  'loc': 'Cagliari',
  'date': '2019-02-21T11:32:05Z',
  'coord': (39.23172, 9.13497),
  'osm_edge': 1521099731},
 7605: {'flux': 45.4,
  'length': 5.2647,
  'conf': 0.89,
  'jam': 2.39552,
  'speed': (32.89, 32.89),
  'loc': 'Quartu S.Elena',
  'date': '2019-02-21T11:32:05Z',
  'coord': (39.24437, 9.18942),
  'osm_edge': 265653413}}

In [4]:
def extract_nodes(traffic_info):
    nodes = {}
    for t, v in traffic_info.items():
        for pc, data in v.items():
            if pc not in nodes:
                crd = data['coord']
                nodes[pc] = {
                    'uuid': str(uuid.uuid4()),
                    'osm_edge': data['osm_edge'], 'lon': crd[1], 'lat': crd[0]}
    return nodes

In [5]:
nodes = extract_nodes(traffic_info_dict)

In [6]:
def define_sensors(nodes):
    sensors = {}
    for pc, d in nodes.items():
        for t in ['jam', 'speed', 'flux', 'length']:
            sensors[(pc, t)] = {'uuid': str(uuid.uuid4()), 'lon': d['lon'], 
                                'lat': d['lat'],
                                'type': t, 'node': d['uuid']}
    return sensors


In [7]:
sensors = define_sensors(nodes)

In [8]:
def dump_nodes(nodes, fname):
    with io.open(fname, 'w') as f:
        w = csv.DictWriter(f, fieldnames=['uuid', 'osm_edge', 'lon', 'lat'])
        w.writeheader()
        for pc, d in nodes.items():
            w.writerow(d)
            
def dump_sensors(sensors, fname):
    with io.open(fname, 'w') as f:
        w = csv.DictWriter(f, fieldnames=['uuid', 'type', 'node', 'lon', 'lat'])
        w.writeheader()
        for pc, d in sensors.items():
            w.writerow(d)

def dump_measures(sensors, traffic, fname):
    with io.open(fname, 'w') as f:
        w = csv.DictWriter(f, fieldnames=['uuid', 'time', 'measure'])
        w.writeheader()
        for t, v in traffic.items():
            for pc, data in v.items():
                for t in ['jam', 'speed', 'flux', 'length']:
                    w.writerow({
                            'uuid': sensors[(pc, t)]['uuid'],
                            'time': data['date'],
                            'measure': data[t][0] if t == 'speed' else data[t]
                            })

In [9]:
dump_nodes(nodes, 'nodes.csv')
dump_sensors(sensors, 'sensors.csv')
dump_measures(sensors, traffic_info_dict, 'measures.csv')